In [876]:
import cv2
# from google.colab.patches import cv2_imshow
import numpy as np

In [877]:
img_width=480
img_height=640

In [878]:
image=cv2.imread('images/4.jpg')

In [879]:
def show_image(image):
  # cv2_imshow(image)
  cv2.imshow('image',image)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

In [880]:
image=cv2.resize(image,(img_width,img_height))
show_image(image)

In [881]:
image_gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
image_blur=cv2.GaussianBlur(image_gray,(5,5),1)

In [882]:
# _, binary_image = cv2.threshold(image_blur, 180, 255, cv2.THRESH_BINARY)

# binary_image = binary_image / 255

# cv2.imshow("Binary Image", binary_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [883]:
show_image(image_blur)

In [884]:
image_canny=cv2.Canny(image_blur,130,255)
kernel=np.ones((3,3))

In [885]:
image_closed=cv2.erode(cv2.dilate(image_canny,kernel,iterations=3),kernel, iterations=1)


In [886]:
show_image(image_closed)

In [887]:
# finding all contours
image_contours=image.copy()
image_f_contours=image.copy()
contours,hierarchy=cv2.findContours(image_closed,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image_contours,contours,-1,(0,255,0),10)

array([[[ 31,  52,  90],
        [ 34,  55,  93],
        [ 32,  53,  91],
        ...,
        [ 11,  31,  66],
        [ 10,  30,  65],
        [ 10,  30,  65]],

       [[ 28,  49,  87],
        [ 32,  53,  91],
        [ 32,  53,  91],
        ...,
        [ 12,  32,  67],
        [ 12,  32,  67],
        [ 11,  31,  66]],

       [[ 27,  48,  86],
        [ 30,  51,  89],
        [ 33,  54,  92],
        ...,
        [ 11,  31,  66],
        [ 11,  31,  66],
        [ 11,  31,  66]],

       ...,

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [ 23,  50,  97],
        [ 25,  53, 100],
        [ 26,  54, 101]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [ 24,  51,  98],
        [ 23,  51,  98],
        [ 31,  59, 106]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [ 18,  45,  92],
        [ 20,  48,  95],
        [ 18,  46,  93]]

In [888]:
show_image(image_contours)

In [889]:
# biggest contours
def find_biggest_contour(contours):
  contours=sorted(contours,key=cv2.contourArea,reverse=True)
  biggest_contour=np.array([])
  max_area=0
  for i in contours:
    area=cv2.contourArea(i)
    if area>4000:
      perimeter=cv2.arcLength(i,True)
      app=cv2.approxPolyDP(i,0.02*perimeter,True)
      if area>max_area and len(app)==4:
        biggest_contour=app
        max_area=area
  return biggest_contour,max_area

In [890]:
biggest_contour,max_area=find_biggest_contour(contours)
biggest_contour

array([[[429,  28]],

       [[ 18,  39]],

       [[  0, 639]],

       [[456, 618]]], dtype=int32)

In [891]:
cv2.drawContours(image_f_contours,biggest_contour,-1,(0,255,0),10)
show_image(image_f_contours)

In [892]:
warped_image=image.copy()
if biggest_contour.size!=0:
  final_points=np.float32([[img_width,0],[0,0],[0 ,img_height],[img_width,img_height]])
  perspective=cv2.getPerspectiveTransform(biggest_contour.astype(np.float32),final_points)
  warped_image=cv2.warpPerspective(image,perspective,(img_width,img_height))


In [893]:
show_image(warped_image)

In [894]:
# warped_image=cv2.cvtColor(warped_image,cv2.COLOR_BGR2GRAY)

In [895]:
# clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
# enhanced = clahe.apply(warped_image)
# binary = cv2.adaptiveThreshold(enhanced, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 5)
# show_image(binary)

In [896]:
# equ = cv2.equalizeHist(warped_image)
# show_image(equ)

In [897]:
# _, binary = cv2.threshold(warped_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
# show_image(binary)

In [898]:
b, g, r = cv2.split(warped_image)
# apply adaptive to color channels
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
blue = clahe.apply(b)
green = clahe.apply(g)
red = clahe.apply(r)
# merge channels into image
equ = cv2.merge((blue, green, red))
show_image(equ)


In [899]:
# make pdf

In [900]:
# make UI